In [62]:
import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lsc
import torch
import h5py
import json
from scipy.spatial.transform import Rotation as R

from deeplsd.utils.tensor import batch_to_device
from deeplsd.models.deeplsd_inference import DeepLSD
# from deeplsd.geometry.viz_2d import plot_images, plot_lines

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R

plt.rcParams.update({'axes.titlesize': 5})

In [63]:
def plot_images(imgs, titles=None, cmaps='gray', dpi=300, size=3, pad=0):
    """Plot a set of images horizontally.
    Args:
        imgs: a list of NumPy or PyTorch images, RGB (H, W, 3) or mono (H, W).
        titles: a list of strings, as titles for each image.
        cmaps: colormaps for monochrome images.
    """
    n = len(imgs)
    if not isinstance(cmaps, (list, tuple)):
        cmaps = [cmaps] * n
    figsize = (size*n, size*3/4) if size is not None else None
    fig, ax = plt.subplots(1, n, figsize=figsize, dpi=dpi)
    if n == 1:
        ax = [ax]
    for i in range(n):
        ax[i].imshow(imgs[i], cmap=plt.get_cmap(cmaps[i]))
        ax[i].get_yaxis().set_ticks([])
        ax[i].get_xaxis().set_ticks([])
        ax[i].set_axis_off()
        for spine in ax[i].spines.values():  # remove frame
            spine.set_visible(False)
        # if titles:
            # ax[i].set_title(titles[i], fontsize=12)
    fig.tight_layout(pad=pad)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)  # Remove spacing

        
        
def plot_lines(lines, line_colors='orange', point_color='cyan',
               ps=1, lw=0.5, indices=(0, 1), alpha=1):
    """ Plot lines and endpoints for existing images.
    Args:
        lines: list of ndarrays of size (N, 2, 2).
        line_colors: string, or list of list of tuples (one for per line).
        point_color: unique color for all endpoints.
        ps: size of the keypoints as float pixels.
        lw: line width as float pixels.
        indices: indices of the images to draw the matches on.
        alpha: alpha transparency.
    """
    if not isinstance(line_colors, list):
        line_colors = [[line_colors] * len(l) for l in lines]
    for i in range(len(lines)):
        if ((not isinstance(line_colors[i], list))
            and (not isinstance(line_colors[i], np.ndarray))):
            line_colors[i] = [line_colors[i]] * len(lines[i])

    fig = plt.gcf()
    # fig.set_size_inches(6, 4) 
    ax = fig.axes
    assert len(ax) > max(indices)
    axes = [ax[i] for i in indices]
    fig.canvas.draw()

    # Plot the lines and junctions
    for a, l, lc in zip(axes, lines, line_colors):
        for i in range(len(l)):
            # x1, x2 = (l[i, 0, 0], l[i, 1, 0])
            # y1, y2 = (l[i, 0, 1], l[i, 1, 1])
            line = matplotlib.lines.Line2D(
                (l[i, 0, 0], l[i, 1, 0]), (l[i, 0, 1], l[i, 1, 1]),
                zorder=1, c=lc[i], linewidth=lw, alpha=alpha)
            a.add_line(line)
        pts = l.reshape(-1, 2)
        a.scatter(pts[:, 0], pts[:, 1], c=point_color, s=ps,
                  linewidths=0, zorder=2, alpha=alpha)     
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)  # Remove spacing
    plt.show()

def shorten_lines(lines, shrink_factor=0.2):
    """
    Shortens all lines by a given percentage from both ends.

    Parameters:
    - lines (Nx2x2 array): Array of lines in the format [[x1, y1], [x2, y2]]
    - shrink_factor (float): Percentage to shorten from both ends (0.2 = 20%)

    Returns:
    - shortened_lines (Nx2x2 array): Array of shortened lines
    """
    if shrink_factor < 0 or shrink_factor > 1:
        raise ValueError("shrink_factor should be between 0 and 1")

    # Convert to float for precision
    lines = np.array(lines, dtype=np.float32)

    # Compute midpoints
    midpoints = (lines[:, 0, :] + lines[:, 1, :]) / 2

    # Move both endpoints closer to the midpoint
    new_endpoints_1 = midpoints + (lines[:, 0, :] - midpoints) * (1 - shrink_factor)
    new_endpoints_2 = midpoints + (lines[:, 1, :] - midpoints) * (1 - shrink_factor)

    # Stack the new endpoints into (N,2,2) shape
    shortened_lines = np.stack([new_endpoints_1, new_endpoints_2], axis=1)

    return shortened_lines.astype(np.int32)  # Convert back to int


def compute_slope(line, epsilon=1e-6):
    """Compute the slope of a line, handling vertical lines with epsilon."""
    (x1, y1), (x2, y2) = line
    return (y2 - y1) / (x2 - x1 + epsilon)  # Avoid division by zero

def compute_length(line):
    """Compute the Euclidean distance (length) of a line."""
    (x1, y1), (x2, y2) = line
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

import cv2
import numpy as np

def check_lines_in_mask(mask_path, lines):
    # Load binary mask image
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Ensure mask is binary (values 0 and 255)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Convert mask to a boolean array (True for inside, False for outside)
    mask_bool = mask > 0  # Pixels inside the mask are True

    # Get image dimensions
    height, width = mask.shape

    inside_both = []
    inside_one = []

    for line in lines:
        (x1, y1), (x2, y2) = line

        # Clip coordinates to ensure they are within image bounds
        x1 = np.clip(x1, 0, width - 1)
        y1 = np.clip(y1, 0, height - 1)
        x2 = np.clip(x2, 0, width - 1)
        y2 = np.clip(y2, 0, height - 1)

        # Check if both endpoints are inside the mask
        is_inside_1 = mask_bool[int(y1), int(x1)]
        is_inside_2 = mask_bool[int(y2), int(x2)]

        if is_inside_1 and is_inside_2:
            inside_both.append([[x1, y1], [x2, y2]])
        elif is_inside_1 or is_inside_2:
            inside_one.append([[x1, y1], [x2, y2]])

    # Convert lists to NumPy arrays with shape (N,2,2) and (M,2,2)
    inside_both = np.array(inside_both, dtype=np.int32) if inside_both else np.empty((0,2,2), dtype=np.int32)
    inside_one = np.array(inside_one, dtype=np.int32) if inside_one else np.empty((0,2,2), dtype=np.int32)

    return inside_both, inside_one

def filter_short_lines(lines, image_height, min_ratio=0.01):
    """
    Remove lines whose length is less than a percentage of the image height.

    Args:
        lines (np.ndarray): Array of shape (N, 2, 2) representing line segments.
        image_height (int): Height of the image.
        min_ratio (float): Minimum length as a fraction of image height (default 1%).

    Returns:
        np.ndarray: Filtered lines with valid lengths.
    """
    if lines.size == 0:
        return lines  # Return empty if no lines exist

    min_length = min_ratio * image_height  # 1% of image height
    lengths = np.linalg.norm(lines[:, 1, :] - lines[:, 0, :], axis=1)  # Compute lengths
    filtered_lines = lines[lengths >= min_length]  # Keep only valid lines

    return filtered_lines

def get_3dpoint_from_moge(img_path, vertices_path, line_points2d):
        '''
        line_points2d.shape = (1,2,2)
        '''
        vertices = np.load(vertices_path)

        mask = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask[:, :] = 0

        this_group_mask = mask.copy()

        for line in line_points2d:
            for point in line:
                # print(point)
                this_group_mask[ int(point[1]), int(point[0])] = 255

        this_group_mask = this_group_mask.reshape(-1,1)

        idx_selected_vertices = np.where(this_group_mask==255)[0]
        # print(idx_selected_vertices)

        selected_vertices = vertices[idx_selected_vertices]

        point1 = selected_vertices[0]
        point2 = selected_vertices[1]
        return point1, point2


def align_plane_to_line(plane_pos_rot, point1, point2):
    '''
    return new plane rotation that is aligned with the line
    '''

    # ✅ Step 1: Define Plane Position & Rotation (Three.js uses radians)
    plane_position = np.array(plane_pos_rot[0])
    plane_rotation = np.radians(plane_pos_rot[1])  # Convert from degrees to radians

    # Apply rotation using SciPy's Rotation module
    rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()

    # Compute the midpoint of the line
    line_midpoint = (point1 + point2) * 0.5

    # Step 4: Compute translation to move the line to the plane's center
    translation = plane_position - line_midpoint
    moved_point1 = point1 + translation
    moved_point2 = point2 + translation

    # Step 6: Calculate angle of the line with the plane's X-axis
    line_direction = (moved_point2 - moved_point1) / np.linalg.norm(moved_point2 - moved_point1)  # Normalize
    plane_x_axis = np.array([1, 0, 0])  # X-axis in local space

    # Rotate the X-axis to align with the plane’s orientation
    rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()
    plane_x_axis_world = np.dot(rotation_matrix, plane_x_axis)

    # Compute the angle using the dot product
    angle_rad = np.arccos(np.clip(np.dot(line_direction, plane_x_axis_world), -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    # Step 1: Define Plane Position & Rotation (Input in Degrees)
    plane_rotation_degrees = plane_pos_rot[1]
    plane_rotation_radians = np.radians(plane_rotation_degrees)  # Convert to radians

    # Compute the transformed X-axis of the plane
    plane_x_axis = np.array([1, 0, 0])  # Local X-axis
    rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()
    plane_x_axis_world = np.dot(rotation_matrix, plane_x_axis)

    # Compute the initial angle
    angle_rad = np.arccos(np.clip(np.dot(line_direction, plane_x_axis_world), -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    # Step 2: Function to compute the new angle after rotating around Z
    def get_final_angle(test_rotation_z):
        test_rotation = plane_rotation.copy()
        test_rotation[2] = test_rotation_z  # Apply test Z-rotation
        test_rotation_matrix = R.from_euler('XYZ', test_rotation).as_matrix()
        test_plane_x_axis = np.dot(test_rotation_matrix, np.array([1, 0, 0]))  # Transformed X-axis
        
        new_angle_rad = np.arccos(np.clip(np.dot(line_direction, test_plane_x_axis), -1.0, 1.0))
        return np.degrees(new_angle_rad)

    # Step 3: Compute both possible Z-rotations
    clockwise_rotation_z = plane_rotation[2] + np.radians(-angle_deg)
    counter_clockwise_rotation_z = plane_rotation[2] + np.radians(angle_deg)

    # Step 4: Compute final angles
    final_angle_clockwise = get_final_angle(clockwise_rotation_z)
    final_angle_counter_clockwise = get_final_angle(counter_clockwise_rotation_z)

    # Step 5: Choose the rotation that minimizes the absolute angle
    if abs(final_angle_clockwise) < abs(final_angle_counter_clockwise):
        best_rotation_z = clockwise_rotation_z
    else:
        best_rotation_z = counter_clockwise_rotation_z

    # Step 7: Convert final plane rotation back to degrees
    plane_rotation_radians[2] = best_rotation_z  # Update Z rotation
    final_plane_rotation_degrees = np.degrees(plane_rotation_radians)  # Convert back to degrees
    

    return final_plane_rotation_degrees


In [64]:
# Model config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = {
    'detect_lines': True,  # Whether to detect lines or only DF/AF
    'line_detection_params': {
        'merge': True,  # Whether to merge close-by lines
        'filtering': True,  # Whether to filter out lines based on the DF/AF. Use 'strict' to get an even stricter filtering
        'grad_thresh': 3,
        'grad_nfa': True,  # If True, use the image gradient and the NFA score of LSD to further threshold lines. We recommand using it for easy images, but to turn it off for challenging images (e.g. night, foggy, blurry images)
    }
}

# Load the model
ckpt = '../weights/deeplsd_md.tar'
ckpt = torch.load(str(ckpt), map_location='cpu')
net = DeepLSD(conf)
net.load_state_dict(ckpt['model'])
net = net.to(device).eval()

In [65]:
DATA_PATH = r"D:\3d-reconstruction\easy-ai-exterior\server\oldui_data\b6fdbad5-9848-4ef1-a28c-bfa3a42092f1"

plane_path = os.path.join(DATA_PATH, "plane.json")
with open(plane_path, "r") as file:
    plane = json.load(file)
print(plane)
print(len(plane.keys()))
num_plane = len(plane.keys())

vertices_path = os.path.join(DATA_PATH, "vertices.npy")
img_path = os.path.join(DATA_PATH, "input.jpg")

{'0_pos_rot': [[-0.16478578746318817, -0.3771667182445526, -1.1906225681304932], [95.38651263243108, 0.5873947376097517, -0.02763178469262397]], '1_pos_rot': [[0.47549647092819214, 0.6072140336036682, -2.2329261302948], [186.31765061968844, 35.625219609546704, -39.480413539940464]]}
2


### Adjust Plane Rotation

In [ ]:
plot = False

new_plane_pos_rot = {}

# Load an image
img = cv2.imread(img_path)[:, :, ::-1]
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Detect (and optionally refine) the lines
inputs = {'image': torch.tensor(gray_img, dtype=torch.float, device=device)[None, None] / 255.}
with torch.no_grad():
    out = net(inputs)
    pred_lines = out['lines'][0]

# # Plot the predictions
if plot:
    plot_images([img], ['All lines'], cmaps='gray')
    plot_lines([pred_lines], indices=range(1))

if pred_lines.size == 0:
    print("no lines detected in this image pred_lines.size == 0")
    new_plane_pos_rot = plane

else:
    # # Shorten the lines
    shortened = shorten_lines(pred_lines, shrink_factor=0.2)
    if plot:
        plot_images([img], ['Shortened lines'], cmaps='gray')
        plot_lines([shortened], indices=range(1))

    for i in range(num_plane):
        plane_key = f"{i}_pos_rot"
        mask_file = f"{i}_binary_mask.jpg"
        mask_path = os.path.join(DATA_PATH, mask_file)

        inside_both, inside_one = check_lines_in_mask(mask_path, shortened)
        
        if inside_both.size != 0:

            if plot:
                plot_images([img], ['Lines within Mask'], cmaps='gray')
                plot_lines([inside_both], indices=range(1))

            # Compute lengths and find the longest line
            lengths = np.array([compute_length(line) for line in inside_both])

            longest_index = np.argmax(lengths)
            longest_line = inside_both[longest_index].reshape(1,2,2)

            if plot:
                plot_images([img], ['Longest line within Mask'], cmaps='gray')
                plot_lines([longest_line], indices=range(1))

            # print(longest_line)

            point1, point2 = get_3dpoint_from_moge(img_path, vertices_path, line_points2d=longest_line)

            this_plane_pos_rot = plane[plane_key]
            print("this_plane_pos_rot", this_plane_pos_rot)


            new_plane_rot = align_plane_to_line(this_plane_pos_rot, point1, point2)
            print("new_plane_rot", new_plane_rot)

            new_plane_pos_rot[plane_key] = [this_plane_pos_rot[0], new_plane_rot.tolist()]

        else: 
            print(f"no line in this mask {mask_path}")
            new_plane_pos_rot[plane_key] = [this_plane_pos_rot[0], this_plane_pos_rot[1]]

        print("="*60)

print("new_plane_pos_rot", new_plane_pos_rot)

this_plane_pos_rot [[-0.16478578746318817, -0.3771667182445526, -1.1906225681304932], [95.38651263243108, 0.5873947376097517, -0.02763178469262397]]
new_plane_rot [ 95.38651263   0.58739474 130.77983323]
this_plane_pos_rot [[0.47549647092819214, 0.6072140336036682, -2.2329261302948], [186.31765061968844, 35.625219609546704, -39.480413539940464]]
new_plane_rot [186.31765062  35.62521961  89.16774384]
new_plane_pos_rot {'0_pos_rot': [[-0.16478578746318817, -0.3771667182445526, -1.1906225681304932], [95.38651263243108, 0.5873947376097517, 130.7798332294909]], '1_pos_rot': [[0.47549647092819214, 0.6072140336036682, -2.2329261302948], [186.31765061968844, 35.625219609546704, 89.1677438373085]]}
